Run the Lesson 3 Code again, but this time do not delete the index in the cleanup phase at the end, as we are using it again here.

In [1]:
! pip install azure-search-documents==11.6.0b4 --quiet
! pip install azure-identity==1.16.1 --quiet
! pip install openai --quiet
! pip install aiohttp --quiet

In [2]:
# Set endpoints and deployment model (provide the name of the deployment)
AZURE_SEARCH_SERVICE: str = "<YOUR_AZURE_SEARCH_SERVICE_URL>"
AZURE_OPENAI_ACCOUNT: str = "<YOUR_AZURE_OPENAI_ACCOUNT_URL>"
AZURE_DEPLOYMENT_MODEL: str = "gpt-35-turbo"

from azure.core.credentials import AzureKeyCredential
credential = AzureKeyCredential("<YOUR_AZURE_AI_SERVICE_KEY>")
INDEX_NAME = "vectest"
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = "<YOUR_AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME>"
EMBEDDING_MODEL_DIMENSIONS = 1024
EMBEDDING_MODEL_NAME = "text-embedding-3-large"

# Set query parameters for grounding the conversation on your search index
search_type="text"
use_semantic_reranker=True
sources_to_include=5

In [3]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchIndex,
)

# credential = AzureKeyCredential(AZURE_SEARCH_SERVICE_API_KEY)
index_client = SearchIndexClient(endpoint=AZURE_SEARCH_SERVICE_URL, credential=credential)

fields = [
    SimpleField(name="ShowId", type=SearchFieldDataType.String, key=True),
    SearchableField(name="ShowName", type=SearchFieldDataType.String, sortable=True),
    SearchableField(
        name="Description", type=SearchFieldDataType.String, analyzer_name="en.lucene"
    ),
    SearchableField(
        name="Category",
        type=SearchFieldDataType.String,
        facetable=True,
        filterable=True,
        sortable=True,
    ),
    SearchableField(
        name="Genres",
        collection=True,
        type=SearchFieldDataType.String,
        facetable=True,
        filterable=True,
    ),
    SimpleField(
        name="Seasons",
        type=SearchFieldDataType.Int32,
        facetable=True,
        filterable=True,
        sortable=True,
    ),
    SimpleField(
        name="Rating",
        type=SearchFieldDataType.Double,
        facetable=True,
        filterable=True,
        sortable=True,
    ),
    SearchableField(
        name="Cast",
        collection=True,
        type=SearchFieldDataType.String,
        facetable=True,
        filterable=True,
    ),
    SearchableField(
        name="StreamingPlatform", type=SearchFieldDataType.String, sortable=True
    ),
    SimpleField(
        name="ReleaseDate",
        type=SearchFieldDataType.DateTimeOffset,
        facetable=True,
        filterable=True,
        sortable=True,
    ),
]

scoring_profiles = []
suggester = [{"name": "sg", "source_fields": ["Genres", "Cast"]}]

Create the Search Index

In [ ]:
from azure.search.documents.indexes.models import (
    SearchIndex,
)

index = SearchIndex(
    name=INDEX_NAME,
    fields=fields,
    suggesters=suggester,
    scoring_profiles=scoring_profiles,
)
result = index_client.create_or_update_index(index)
print(f' {result.name} created')

In [ ]:
# Set up the query for generating responses
from azure.identity import DefaultAzureCredential
from azure.identity import get_bearer_token_provider
from azure.search.documents import SearchClient
from openai import AzureOpenAI

# credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")
openai_client = AzureOpenAI(
    api_version="2024-06-01",
    azure_endpoint=AZURE_OPENAI_ACCOUNT,
    azure_ad_token_provider=token_provider
)

search_client = SearchClient(
    endpoint=AZURE_SEARCH_SERVICE,
    index_name="vectest",
    credential=credential
)

# This prompt provides instructions to the model
GROUNDED_PROMPT="""
You are a friendly assistant that recommends television shows.
Answer the query using only the sources provided below in a friendly and concise bulleted manner.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
Query: {query}
Sources:\n{sources}
"""

# Query is the question being asked. It's sent to the search engine and the LLM.
query="Can you recommend a funny show about a group of friends?"

# Set up the search results and the chat thread.
# Retrieve the selected fields from the search index related to the question.
search_results = search_client.search(
    search_text=query,
    top=5,
    select="title,content,category"
)
sources_formatted = "\n".join([f'{document["title"]}:{document["content"]}:{document["category"]}' for document in search_results])

response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": GROUNDED_PROMPT.format(query=query, sources=sources_formatted)
        }
    ],
    model=AZURE_DEPLOYMENT_MODEL
)

print(response.choices[0].message.content)